In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
from datetime import date
from datetime import datetime
import unidecode
import re

%matplotlib inline

In [26]:
all_players = pd.read_csv('../ChatGPT/gpt_food.csv')
players_stats = pd.read_csv('../ChatGPT/player_stats.csv')
#current_squad = pd.read_csv('../ChatGPT/current_squad.csv')

In [4]:
all_players.head(2)

,Unnamed: 0,player_id,fifa_version,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,0,231747,24,k. mbappe,Kylian Mbappé Lottin,"ST, LW",91,94,181500000.0,230000.0,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,1,239085,24,e. haaland,Erling Braut Haaland,ST,91,94,185000000.0,340000.0,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3


In [5]:
players_stats.head(2)

,Unnamed: 0,rank,player_name,nationality,team,position,overall_rating,pace,shooting,passing,dribbling,defense,physical
0,0,1,Kylian Mbappe,NaN,NaN,ST,91,97,90,80,92,36,78
1,1,2,Alexia Putellas,NaN,NaN,CM,91,82,90,91,92,72,78


In [30]:
#convert to lower case and drop random unneeded row
players_stats['player_name'] = players_stats['player_name'].str.lower()
players_stats = players_stats.drop(columns = ['Unnamed: 0'])

#split first and last name
players_stats[['first_name', 'last_name']] = players_stats['player_name'].str.split(' ', 1, expand = True)

#combine into short_name
players_stats['short_name'] = players_stats.apply(lambda row: f"{row['first_name'][0]}. {row['last_name']}", axis = 1)
players_stats

,rank,player_name,nationality,team,position,overall_rating,pace,shooting,passing,dribbling,defense,physical,first_name,last_name,short_name
0,1,kylian mbappe,NaN,NaN,ST,91,97,90,80,92,36,78,kylian,mbappe,k. mbappe
1,2,alexia putellas,NaN,NaN,CM,91,82,90,91,92,72,78,alexia,putellas,a. putellas
2,3,erling haaland,NaN,NaN,ST,91,89,93,66,80,45,88,erling,haaland,e. haaland
3,4,kevin de bruyne,NaN,NaN,CM,91,72,88,94,87,65,78,kevin,de bruyne,k. de bruyne
4,5,aitana bonmati,NaN,NaN,CM,90,81,84,83,91,75,73,aitana,bonmati,a. bonmati
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17321,17318,deven sawhney,NaN,NaN,LB,47,62,26,34,42,43,57,deven,sawhney,d. sawhney
17322,17318,wang tengda,NaN,NaN,CM,47,61,37,45,48,43,47,wang,tengda,w. tengda
17323,17318,wu yuhang,NaN,NaN,CDM,47,56,34,43,45,43,51,wu,yuhang,w. yuhang
17324,17318,ye daoxin,NaN,NaN,CB,47,58,28,33,39,49,48,ye,daoxin,y. daoxin


In [93]:
#pivoted from creating scratch team because it appears that players are made up
test_squad = all_players[all_players['club_name'].str.contains('Argyle', na = False)]
test_squad = test_squad[['short_name', 'long_name', 'player_positions', 'value_eur', 'wage_eur']]

## for some reason this didn't work
#test_squad[['first_name', 'last_name']] = test_squad['long_name'].str.split(' ', expand=True).iloc[:, [0,-1]]


test_squad['first_name'] = test_squad['long_name'].str.split().str[0]
test_squad['last_name'] = test_squad['long_name'].str.split().str[-1]
test_squad['first_name'] = test_squad['first_name'].str.lower()
test_squad['last_name'] = test_squad['last_name'].str.lower()
test_squad

,short_name,long_name,player_positions,value_eur,wage_eur,first_name,last_name
3630,m. cooper,Michael John Cooper,GK,3600000.0,7000.0,michael,cooper
3633,b. mumba,Bali Mumba,"LM, LWB, RWB",4200000.0,9000.0,bali,mumba
5347,f. azaz,Finn Isaac Azaz,"RM, LM, CAM",3100000.0,27000.0,finn,azaz
5636,pleguezuelo,Julio José Pleguezuelo Selva,CB,1500000.0,8000.0,julio,selva
5718,d. scarr,Daniel George Scarr,CB,1300000.0,8000.0,daniel,scarr
6040,m. gillesphey,Macaulay Gillesphey,CB,1300000.0,8000.0,macaulay,gillesphey
6790,m. bundu,Mustapha Bundu,"RW, RM",1400000.0,8000.0,mustapha,bundu
6872,j. edwards,Joseph Robert Edwards,"RWB, LWB, RB",825000.0,7000.0,joseph,edwards
8458,l. cundle,Luke James Cundle,"CM, CAM, RW",1900000.0,12000.0,luke,cundle
8500,c. wright,Callum Wright,CM,1900000.0,5000.0,callum,wright


In [94]:
test_squad = test_squad.rename(columns = {'long_name':'player_name'})
test_squad = pd.merge(test_squad, players_stats, on = ['last_name', 'first_name'])
test_squad

,short_name_x,player_name_x,player_positions,value_eur,wage_eur,first_name,last_name,rank,player_name_y,nationality,team,position,overall_rating,pace,shooting,passing,dribbling,defense,physical,short_name_y
0,m. cooper,Michael John Cooper,GK,3600000.0,7000.0,michael,cooper,4282,michael cooper,NaN,NaN,GK,71,70,69,67,72,32,69,m. cooper
1,b. mumba,Bali Mumba,"LM, LWB, RWB",4200000.0,9000.0,bali,mumba,4282,bali mumba,NaN,NaN,LWB,71,84,60,65,73,64,58,b. mumba
2,f. azaz,Finn Isaac Azaz,"RM, LM, CAM",3100000.0,27000.0,finn,azaz,5946,finn azaz,NaN,NaN,RM,69,74,64,66,69,57,68,f. azaz
3,m. gillesphey,Macaulay Gillesphey,CB,1300000.0,8000.0,macaulay,gillesphey,5946,macaulay gillesphey,NaN,NaN,CB,69,53,41,53,57,68,77,m. gillesphey
4,l. cundle,Luke James Cundle,"CM, CAM, RW",1900000.0,12000.0,luke,cundle,8978,luke cundle,NaN,NaN,CM,66,67,58,65,66,52,48,l. cundle
5,c. wright,Callum Wright,CM,1900000.0,5000.0,callum,wright,8978,callum wright,NaN,NaN,CM,66,67,61,64,67,57,67,c. wright
6,m. whittaker,Morgan Whittaker,"RM, RW",1900000.0,5000.0,morgan,whittaker,8978,morgan whittaker,NaN,NaN,RM,66,74,65,58,67,38,60,m. whittaker
7,a. randell,Adam Fletcher Randell,CM,1700000.0,5000.0,adam,randell,8978,adam randell,NaN,NaN,CM,66,76,62,65,69,63,69,a. randell
8,j. houghton,Jordan Alexander Houghton,"CDM, CM",925000.0,5000.0,jordan,houghton,8978,jordan houghton,NaN,NaN,CDM,66,59,51,64,61,62,75,j. houghton
9,r. hardie,Ryan Hardie,ST,1100000.0,6000.0,ryan,hardie,8978,ryan hardie,NaN,NaN,ST,66,86,64,48,64,24,61,r. hardie


In [71]:
players_stats

,rank,player_name,nationality,team,position,overall_rating,pace,shooting,passing,dribbling,defense,physical,first_name,last_name,short_name
0,1,kylian mbappe,NaN,NaN,ST,91,97,90,80,92,36,78,kylian,mbappe,k. mbappe
1,2,alexia putellas,NaN,NaN,CM,91,82,90,91,92,72,78,alexia,putellas,a. putellas
2,3,erling haaland,NaN,NaN,ST,91,89,93,66,80,45,88,erling,haaland,e. haaland
3,4,kevin de bruyne,NaN,NaN,CM,91,72,88,94,87,65,78,kevin,de bruyne,k. de bruyne
4,5,aitana bonmati,NaN,NaN,CM,90,81,84,83,91,75,73,aitana,bonmati,a. bonmati
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17321,17318,deven sawhney,NaN,NaN,LB,47,62,26,34,42,43,57,deven,sawhney,d. sawhney
17322,17318,wang tengda,NaN,NaN,CM,47,61,37,45,48,43,47,wang,tengda,w. tengda
17323,17318,wu yuhang,NaN,NaN,CDM,47,56,34,43,45,43,51,wu,yuhang,w. yuhang
17324,17318,ye daoxin,NaN,NaN,CB,47,58,28,33,39,49,48,ye,daoxin,y. daoxin


In [81]:
test_squad.dtypes

short_name           object
long_name            object
player_positions     object
value_eur           float64
wage_eur            float64
first_name           object
last_name            object
dtype: object